In [1]:
import pandas as pd
from pysam import FastaFile
import json
import logging
from crc64iso import crc64iso

In [2]:
logger = logging.getLogger(__name__)

In [3]:
protein_fa = FastaFile('../tracked_results/gencode_34_unique_protein_seq.fasta.gz')

In [4]:
all_prot_entries = protein_fa.references
len(all_prot_entries)

87065

In [5]:
all_prot_entries[:5]

['1', '2', '3', '4', '5']

In [6]:
%%time
df = pd.DataFrame({'ensembl_prot_ids': all_prot_entries})
df['uniparc_checksum'] = df['ensembl_prot_ids'].apply(protein_fa.fetch).apply(crc64iso.crc64)

CPU times: user 12 s, sys: 186 ms, total: 12.1 s
Wall time: 12.2 s


In [7]:
with open('../../intermediates/gencode_34/uniparc_aria2_downloads.links', 'w') as f:
    for t in df.itertuples(index=False):
        print(
            f'https://www.uniprot.org/uniparc/?query=checksum%3A{t.uniparc_checksum}&format=xml\n'
            f'  out=xmls/{t.ensembl_prot_ids}.xml',
            file=f
        )